# Statistical foundations of machine learning - <span style="color:#1CA766">INFO-F-422</span>
# Project: <span style="color:#1CA766">House Prices: Advanced Regression Techniques</span>

> ## <span style="color:#2E66A7"> Alberto Parravicini</span>

*****

# <span style="color:#1CA766">Part 7:</span> Model building and predictions

In the first part of the report we have explored in details our dataset, and seen how to preprocess it to extract as much information as possible from it. 

Now, we'll put to use all of the previous hard work, and see how to get some high quality predictions for the houses!

***

I've tried quite a large number of different models on the dataset, from **decision trees** to **K-nearest neighbour**, in order to see which one would work better.  

At the end of the day, the best results were obtained by **ridge regression** and **SVM**, with their predictions combined by a simple **neural network** to get the highest quality predictions.

To keep thing short, I won't show all the experiments with different models and parameters, but they are all available in the *src* folder.

***

For the models that were considered, the approach that was followed is always the same, and is summarized below.

    
    
    # Initialization
    
    * train = load the training set
    * test = load the test set
       
    * Build the index for cross-validation (always the same for the different parameters, so the results are comparable)
    * grid = prepare the grid of parameters that will be tested on the model
    
    
    
    # Cross-validation to find the best parameters
    
    * for each parameter p in the grid:
        * for each fold f of cross-validation:
            * Build the cross-validation train set and test set, by using the index
            * Normalize the cross-validation training and test set, so we don't have leaks
            
            * Train the model on the cross-validation training set, with the current parameter p
            * Test the model on the cross-validation test set
            
            * Scale back and round the predictions
            * Store the predictions on a file (they will be used to train the ensemble model)
        
        * Compute the average RMSE across the cross-validation, for the current parameter p
        
    
    
    # Final predictions
    
    * Scale the training set and the test set
    
    * Train the model with the entire training set, with the best parameters found above
    * Predict the values for the test set
    
    * Scale back the predictions
    * Round the predictions
    * Save the predictions
    
    * Profit!
            
            
       
    
    
    
    

A few notes:

* The train set and the test set are **scaled** at **each step** of cross-validation, so that we are sure that they are properly normalized, and we have no leaks from the test set to the train set (well, unless we use the dataset where we leaked information on purpose!)
* The test set is obviously scaled using the means and variances computed on the training set.
* The cross-validation folds have to be **the same** for all the parameters we are testing, otherwise the results we obtain aren't comparable! Depending on how we sample the fold, we might get different results!
* After building the cross-validation training set, it might happen that some of its features have $0$ variance. We must be careful to remove those!
* The **RMSE** is computed on the results after they have been scaled back, but before computing the exponential (remember that we applied the $log$ transformation to the prices!).  
This is because in the leaderboard it is computed the **RMSE** on the logarithms!
* All the prices in the dataset are multiple of $100$, and most are multiple of $500$. **Rounding** the results to the closest multiple of $500$ gives a good boost in accuracy!
* The predictions done during crossvalidations are stored in a file and are used to train the ensemble model. Of course, only the predictions done with the best parameters are actually used in the training, the others are discarded.
* The **neural network** used for ensembling works also in a similar way, but we'll talk about it more in details later.

***

## <span style="color:#2E66A7"> Ridge regression</span>

The first model we are going to use is **ridge regression**, which is a linear model with an additional penalty factor that is proportional to the weights of the model.  
The idea is that ridge regression will try to keep the weights as small as possible, in order to reduce the variance of the model and make it less prone to overfitting. Formally, we will try to compute the quantity:

$$\arg\min_{\beta,\ \lambda} (y - X\beta)^T(y - X\beta) + \lambda\lVert\beta\rVert^2_2$$

where $\lambda$ is the penalty coefficient assigned to the weights, and is multiplied by the $L2$ norm of the weights.

We will try different values of $\lambda$, and assess the optimal value by using cross-validation.

In [80]:
rm(list=ls())

library(readr)
library(purrr)
library(tidyr)
library(rbokeh)
library(plyr)
library(tidyr)
library(ggplot2)
library(moments)
library(sqldf)
library(xtable)
library(dplyr)
library(psych)
library(corrplot)
library(infotheo)
library(ridge)
library(Metrics)
library(reshape)



#############
# READ DATA #
#############

# Import the data (the scaled/leaky version, it scores better!)
train <- read_csv("../data/train_fin.csv")
test <- read_csv("../data/test_fin.csv")

all_data <- rbind(train[, -ncol(train)], test)

# Print header
print(head(train))
# Count column types
table(sapply(train, class))

Warning message:
"package 'reshape' was built under R version 3.3.3"
Attaching package: 'reshape'

The following object is masked from 'package:dplyr':

    rename

The following objects are masked from 'package:plyr':

    rename, round_any

The following objects are masked from 'package:tidyr':

    expand, smiths

Parsed with column specification:
cols(
  .default = col_double(),
  Id = col_integer()
)
See spec(...) for full column specifications.
Parsed with column specification:
cols(
  .default = col_double(),
  Id = col_integer()
)
See spec(...) for full column specifications.


# A tibble: 6 × 296
     Id LotFrontage    LotArea      Street LandSlope OverallQual OverallCond
  <int>       <dbl>      <dbl>       <dbl>     <dbl>       <dbl>       <dbl>
1     1 -0.18902161 -0.1010474 -0.06427141 0.2245328  0.64962667  -0.4351594
2     2  0.48866733  0.1502778 -0.06427141 0.2245328 -0.06118435   1.9058002
3     3 -0.05348382  0.4626843 -0.06427141 0.2245328  0.64962667  -0.4351594
4     4 -0.41491792  0.1399921 -0.06427141 0.2245328  0.64962667  -0.4351594
5     5  0.66938438  0.9296924 -0.06427141 0.2245328  1.36043769  -0.4351594
6     6  0.71456364  0.9095607 -0.06427141 0.2245328 -0.77199537  -0.4351594
# ... with 289 more variables: YearBuilt <dbl>, YearRemodAdd <dbl>,
#   RoofMatl <dbl>, MasVnrArea <dbl>, ExterQual <dbl>, ExterCond <dbl>,
#   BsmtQual <dbl>, BsmtCond <dbl>, BsmtExposure <dbl>, BsmtFinType1 <dbl>,
#   BsmtFinSF1 <dbl>, BsmtFinType2 <dbl>, BsmtFinSF2 <dbl>, BsmtUnfSF <dbl>,
#   TotalBsmtSF <dbl>, Heating <dbl>, HeatingQC <dbl>, CentralAir <dbl>


integer numeric 
      1     295 

First, we load the (unscaled) data. Then we prepare the dataset for cross-validation.

In [30]:
################
# PREPARE DATA #
################

# shuffle dataset
train <- train[sample(nrow(train)),]

x_train = train[, 2:ncol(train)]
y_train = train$SalePrice

x_test = test[, 2:ncol(test)]

num_folds = 10

# Save the predictions done by crossvalidation
xval_pred <- data.frame(Id=c(), SalePrice=c())

Then we apply cross-validation to find the optimal value of $\lambda$. 

It turns out that the best value is found for $\lambda = 20$ (even if cross validation says otherwise!).

In [32]:
#########
# RIDGE #
#########

   
index = sample(1:nrow(x_train))
fold_size <- floor(nrow(x_train) / num_folds)

cors = c(1)
lambdas = c(1, 10, 20, 40, 50)

for (l in lambdas)
{
  cv_errors <- NULL
  
  for (i in 1:num_folds)
  {
    index_test <- (((i-1) * fold_size + 1):(i * fold_size))  
    # Make sure that all the examples are tested.
    if (i==num_folds)
    {
      index_test = c(index_test, ((i * fold_size)+1):nrow(x_train))
    }
    index_train <- setdiff(index, index_test)
    
    X_tr <- x_train[index_train, ]
    Y_tr <- y_train[index_train]  
    X_val <- x_train[index_test, ]  
    Y_val <- y_train[index_test] 
    
    # Scale x
    x_mean <- sapply(X_tr, mean)
    x_sd <- sapply(X_tr, sd)
    # Remove rows with variance = 0
    var_zero <- x_sd == 0
    X_tr <- data.frame(scale(X_tr, center = x_mean, scale = x_sd))
    X_tr[, var_zero] <- NULL
    X_val <- data.frame(scale(X_val, center = x_mean, scale = x_sd))
    X_val[, var_zero] <- NULL
    
    
    # Scale the output.
    Y_mean <- mean(Y_tr)
    Y_sd <- sd(Y_tr)
    Y_tr <- scale(Y_tr, center = Y_mean, scale = Y_sd)
    Y_val <- scale(Y_val, center = Y_mean, scale = Y_sd)
    
    
    model<- linearRidge(SalePrice ~., data = X_tr, scaling = "none", lambda = l)
    
    pred <- predict(model, X_val)
    cv_errors <- c(cv_errors, rmse(pred * Y_sd + Y_mean, Y_val * Y_sd + Y_mean))
    
    # Scale back the predictions.
    pred <- pred * Y_sd + Y_mean
    pred <- exp(pred) - 1
    # Round to the closest 500
    pred <- round(pred / 500) * 500
    
    pred_data_temp <- data.frame(Id=train[index_test, ]$Id, SalePrice=pred)
    # Add the new predictions
    xval_pred <- rbind(xval_pred, pred_data_temp)
  }
    
  print(paste("LAMBDA=", l, "; CV error=", round(mean(cv_errors), digits=4), " ; std dev=", round(sd(cv_errors), digits=4)))
}

xval_pred <- xval_pred %>% arrange(Id)
# Store the cross validation predictions, used to train the ensemble.
# write.csv(x=xval_pred, file="../data/predictions/ridge_xval.csv", row.names=F)

[1] "LAMBDA= 1 ; CV error= 0.1153  ; std dev= 0.0194"
[1] "LAMBDA= 10 ; CV error= 0.1136  ; std dev= 0.0195"
[1] "LAMBDA= 20 ; CV error= 0.1132  ; std dev= 0.0195"
[1] "LAMBDA= 40 ; CV error= 0.113  ; std dev= 0.0194"
[1] "LAMBDA= 50 ; CV error= 0.1129  ; std dev= 0.0193"


In [33]:
###########
# PREDICT #
###########

Y_mean <- mean(y_train)
Y_sd <- sd(y_train)
x_train$SalePrice <- scale(x_train$SalePrice, center = Y_mean, scale = Y_sd)

model<- linearRidge(SalePrice ~., data = x_train, scaling = "none", lambda = 20)

pred <- predict(model, x_test)


# Scale back the prediction.
pred <- pred * Y_sd + Y_mean
pred <- exp(pred) - 1
# Round to the closest 500
pred <- round(pred / 500) * 500

pred_data <- data.frame(Id=test$Id, SalePrice=pred)

# Store the final predictions.
# write.csv(x=pred_data, file="../data/predictions/ridge_fin_01.csv", row.names=F)

With just this basic model we are already able to get a reasonable score of $0.12142$ (which is a bit higher that what is told by cross validation, but it is still quite good).

***

Now, let's try to get some insight about what we have done, as it might be useful to improve our models!

First, let's look at the **coefficients** of the model. 
The nice thing of linear models is that each coefficient represents the *importance* of a given feature, it's degree of association with the prediction.  
If we look at the coefficients we can understand which features are more important, and which are considered useless.

In [34]:
coef_df <- data.frame(x=rownames(model$coef), y=model$coef)
p <- figure(width=1200, height=800) %>% ly_bar(x, y, data=coef_df) %>%
  theme_axis(major_label_orientation = 45) %>%
  x_range(dat=coef_df[order(coef_df$y, decreasing = T), ]$x)

rbokeh2html(p,'./out.html')
IRdisplay::display_html(file='./out.html')

html file written to: ./out.html


<!DOCTYPE html>

Ok, this is a mess! 

Let's keep only the most significant coefficients.

In [35]:
p <- figure() %>% ly_bar(x, y, data=subset(coef_df, abs(y) > 0.04)) %>%
  theme_axis(major_label_orientation = 45) %>%
  x_range(dat=subset(coef_df, abs(y) > 0.04)[order(subset(coef_df, abs(y) > 0.04)$y, decreasing = T), ]$x)

rbokeh2html(p,'./out.html')
IRdisplay::display_html(file='./out.html')

html file written to: ./out.html


<!DOCTYPE html>

The **area** and the overall **quality** of the house are extremely important. Moreover, we see that having a newer house also affects the price, as the coefficient of **age** is negative.  
Most importantly, commercial buildings are cheaper!

We can see from the first plot how most coefficients are small, but still different from 0.  
Using **Lasso** regression would have kept most of them to zero, giving a sightly more interpretable model (but worse results, I've tried!)

We can also look at how different the predictions on the train set are different from the real data, to see if there are specific cases in which our model performs badly.

In [42]:
xval_pred <- read.csv("../data/predictions/ridge_xval.csv")
train <- train %>% arrange(Id)
p <- figure(legend_location = "top_left") %>% ly_points(x=train$SalePrice, y = log(1 + xval_pred$SalePrice), data=train, hover=Id) %>%
  ly_abline(lm(log(1 + xval_pred$SalePrice) ~ train$SalePrice, data=train), width=2, type=2, legend="Predicted") %>%
  ly_abline(a = 0, b = 1, width=2, type=2, color = "red", legend="Ideal")

rbokeh2html(p,'./out.html')
IRdisplay::display_html(file='./out.html')

html file written to: ./out.html


<!DOCTYPE html>

We can see how our model performs badly especially for low price houses. House $89$ is especially problematic, as its predicted price is much lower than the real one.  
We can see that the house is of type **commercial**, that it has low quality and all the worst possible things, and yet its price is higher than expected.

Could removing that house from the dataset be beneficial?  
It turns out that it's not, and just removing that single exampel lowers our score by $0.01$, enough to lose 50 positions on the leaderboard!

***

## <span style="color:#2E66A7"> SVM</span>

Let's move to a sightly fancier model!  
Here we are going to use **Support vector machines**, and see if a more complex model can give us some improvement over the previous one.

The process of training the **SVM** is exactly the same as the **ridge regression**, but there are many more parameters to chose, some of which with a rather obscure meaning.  
We have to set the penalization for the *soft-margin*, the type of *kernel*, the parameters of the *kernel*, the parameters of the *optimization algorithm*, and more!

Even after all the parameter tuning, we get a model which gives about the same performances of **ridge regression**.  
Still, having a second high performing model will be good for our final ensemble model!  

*The more, the merrier!*

In [44]:
rm(list=ls())

library(readr)
library(purrr)
library(tidyr)
library(rbokeh)
library(plyr)
library(tidyr)
library(ggplot2)
library(moments)
library(sqldf)
library(xtable)
library(dplyr)
library(psych)
library(corrplot)
library(infotheo)
library(e1071)
library(Metrics)


#############
# READ DATA #
#############

# Import the data
train <- read_csv("../data/train_fin.csv")
test <- read_csv("../data/test_fin.csv")

all_data <- rbind(train[, -ncol(train)], test)

# Print header
print(head(train))
# Count column types
table(sapply(train, class))


################
# PREPARE DATA #
################

# shuffle dataset
train <- train[sample(nrow(train)),]

x_train = train[, 2:ncol(train)]
y_train = train$SalePrice

x_test = test[, 2:ncol(test)]

num_folds = 10

# Save the predictions done by crossvalidation
xval_pred <- data.frame(Id=c(), SalePrice=c())


Attaching package: 'e1071'

The following objects are masked from 'package:moments':

    kurtosis, moment, skewness

Parsed with column specification:
cols(
  .default = col_double(),
  Id = col_integer()
)
See spec(...) for full column specifications.
Parsed with column specification:
cols(
  .default = col_double(),
  Id = col_integer()
)
See spec(...) for full column specifications.


# A tibble: 6 × 296
     Id LotFrontage    LotArea      Street LandSlope OverallQual OverallCond
  <int>       <dbl>      <dbl>       <dbl>     <dbl>       <dbl>       <dbl>
1     1 -0.18902161 -0.1010474 -0.06427141 0.2245328  0.64962667  -0.4351594
2     2  0.48866733  0.1502778 -0.06427141 0.2245328 -0.06118435   1.9058002
3     3 -0.05348382  0.4626843 -0.06427141 0.2245328  0.64962667  -0.4351594
4     4 -0.41491792  0.1399921 -0.06427141 0.2245328  0.64962667  -0.4351594
5     5  0.66938438  0.9296924 -0.06427141 0.2245328  1.36043769  -0.4351594
6     6  0.71456364  0.9095607 -0.06427141 0.2245328 -0.77199537  -0.4351594
# ... with 289 more variables: YearBuilt <dbl>, YearRemodAdd <dbl>,
#   RoofMatl <dbl>, MasVnrArea <dbl>, ExterQual <dbl>, ExterCond <dbl>,
#   BsmtQual <dbl>, BsmtCond <dbl>, BsmtExposure <dbl>, BsmtFinType1 <dbl>,
#   BsmtFinSF1 <dbl>, BsmtFinType2 <dbl>, BsmtFinSF2 <dbl>, BsmtUnfSF <dbl>,
#   TotalBsmtSF <dbl>, Heating <dbl>, HeatingQC <dbl>, CentralAir <dbl>


integer numeric 
      1     295 

In [46]:
#######
# SVM #
#######

index = sample(1:nrow(x_train))
fold_size <- floor(nrow(x_train) / num_folds)

# Here I list only few of the parameters that were tried, as an example.
params <- expand.grid(g = 2^(-12),
                      c = c(5, 7, 10),
                      t= c("nu-regression"),
                      n = c(0.7, 0.9),
                      e = c(0.1),
                      k = c("radial"),
                      c0 = c(1.2),
                      d = c(5))


for (i in 1:nrow(params))
{
  #############
  #############
  p <- params[i, ]
  
  cv_errors <- NULL
  for (i in 1:num_folds) {
    
    index_test <- (((i-1) * fold_size + 1):(i * fold_size))  
    # Make sure that all the examples are tested.
    if (i==num_folds)
    {
      index_test = c(index_test, ((i * fold_size)+1):nrow(x_train))
    }
    index_train <- setdiff(index, index_test)
    
    X_tr <- x_train[index_train, ]
    Y_tr <- y_train[index_train]  
    X_val <- x_train[index_test, ]  
    Y_val <- y_train[index_test] 
    
    # Scale x
    x_mean <- sapply(X_tr, mean)
    x_sd <- sapply(X_tr, sd)
    # Remove features with variance = 0
    var_zero <- x_sd == 0
    X_tr <- data.frame(scale(X_tr, center = x_mean, scale = x_sd))
    X_tr[, var_zero] <- NULL
    X_val <- data.frame(scale(X_val, center = x_mean, scale = x_sd))
    X_val[, var_zero] <- NULL


    # Scale the output.
    Y_mean <- mean(Y_tr)
    Y_sd <- sd(Y_tr)
    Y_tr <- scale(Y_tr, center = Y_mean, scale = Y_sd)
    Y_val <- scale(Y_val, center = Y_mean, scale = Y_sd)
    
    
    model<- svm(SalePrice ~ ., data = X_tr,
                gamma=p$g,
                cost=p$c,
                type="nu-regression",
                nu=p$n,
                epsilon=p$e,
                kernel=p$k,
                coef0=p$c0,
                degree=p$d)
    
    pred <- predict(model, X_val)
    cv_errors <- c(cv_errors, rmse(pred * Y_sd + Y_mean, Y_val * Y_sd + Y_mean))
    
    
    # Scale back the prediction.
    pred <- pred * Y_sd + Y_mean
    pred <- exp(pred) - 1
    # Round to the closest 500
    pred <- round(pred / 500) * 500
    
    pred_data_temp <- data.frame(Id=train[index_test, ]$Id, SalePrice=pred)
    # Add the new predictions
    xval_pred <- rbind(xval_pred, pred_data_temp)
  }
  
  print(paste("DEGREE=", p$d, "; COEF0=", p$c0, "; KERNEL=", p$k, "; COST=", p$c, "; GAMMA=", p$g, "; NU=", p$n, "; CV error=", round(mean(cv_errors), digits=4), " ; std dev=", round(sd(cv_errors), digits=4)))
  #############
  #############
}


xval_pred <- xval_pred %>% arrange(Id)
# Store the cross validation predictions, used to train the ensemble.
# write.csv(x=xval_pred, file="../data/predictions/svm_xval.csv", row.names=F)

[1] "DEGREE= 5 ; COEF0= 1 ; KERNEL= radial ; COST= 5 ; GAMMA= 0.000244140625 ; NU= 0.7 ; CV error= 0.1131  ; std dev= 0.0099"
[1] "DEGREE= 5 ; COEF0= 1 ; KERNEL= radial ; COST= 7 ; GAMMA= 0.000244140625 ; NU= 0.7 ; CV error= 0.1124  ; std dev= 0.0101"
[1] "DEGREE= 5 ; COEF0= 1 ; KERNEL= radial ; COST= 10 ; GAMMA= 0.000244140625 ; NU= 0.7 ; CV error= 0.1113  ; std dev= 0.0103"
[1] "DEGREE= 5 ; COEF0= 1.2 ; KERNEL= radial ; COST= 5 ; GAMMA= 0.000244140625 ; NU= 0.7 ; CV error= 0.1131  ; std dev= 0.0099"
[1] "DEGREE= 5 ; COEF0= 1.2 ; KERNEL= radial ; COST= 7 ; GAMMA= 0.000244140625 ; NU= 0.7 ; CV error= 0.1124  ; std dev= 0.0101"
[1] "DEGREE= 5 ; COEF0= 1.2 ; KERNEL= radial ; COST= 10 ; GAMMA= 0.000244140625 ; NU= 0.7 ; CV error= 0.1113  ; std dev= 0.0103"
[1] "DEGREE= 5 ; COEF0= 1.4 ; KERNEL= radial ; COST= 5 ; GAMMA= 0.000244140625 ; NU= 0.7 ; CV error= 0.1131  ; std dev= 0.0099"
[1] "DEGREE= 5 ; COEF0= 1.4 ; KERNEL= radial ; COST= 7 ; GAMMA= 0.000244140625 ; NU= 0.7 ; CV error= 0.1124 

The best parameters that were found are the following:
* **Radial** kernel.
* **Cost** $=7$, cost of the *soft-margin* violation.
* $\gamma = 2^{-12}$, a parameter of the kernel function.
* **Degree** $=5$, another parameter of the kernel (unused).
* **Coefficient 0** $=1.2$, another parameter of the kernel (unused).
* Regression of type $\nu$.
* $\nu = 0.7$, parameter of the regression.
* $\varepsilon = 0.1$, parameter of the optimizer.

In [ ]:
###########
# PREDICT #
###########

Y_mean <- mean(y_train)
Y_sd <- sd(y_train)
x_train$SalePrice <- scale(x_train$SalePrice, center = Y_mean, scale = Y_sd)

model <- svm(SalePrice ~ ., data = x_train, gamma=2^-12, cost=7, nu=0.7, type="nu-regression")

pred <- predict(model, x_test)

# Scale back the prediction.
pred <- pred * Y_sd + Y_mean
pred <- exp(pred) - 1
# Round to the closest 500
pred <- round(pred / 500) * 500

pred_data <- data.frame(Id=test$Id, SalePrice=pred)

# write.csv(x=pred_data, file="../data/predictions/svm_fin.csv", row.names=F)

With this model we are able to get a score of $0.12143$, which is not an improvement over the previous one!  
We need to resort to some ensembling to do even better!

***

Anyway, let's look at the predictions given by this model, to see if it performs differently from the previous one, even though the score is the same.

In [52]:
xval_pred_svm <- read.csv("../data/predictions/svm_xval.csv")
train <- train %>% arrange(Id)
p <- figure(legend_location = "top_left") %>% ly_points(x=train$SalePrice, y = log(1 + xval_pred_svm$SalePrice), data=train, hover=Id) %>%
  ly_abline(lm(log(1 + xval_pred_svm$SalePrice) ~ train$SalePrice, data=train), width=2, type=2, legend="Predicted") %>%
  ly_abline(a = 0, b = 1, width=2, type=2, color = "red", legend="Ideal")

rbokeh2html(p,'./out.html')
IRdisplay::display_html(file='./out.html')

html file written to: ./out.html


<!DOCTYPE html>

In [71]:
xval_pred_svm <- read.csv("../data/predictions/svm_xval.csv")
xval_pred_rlm <- read.csv("../data/predictions/ridge_xval.csv")

train <- train %>% arrange(Id)
p <- figure(legend_location = "top_left", width=400) %>% 
    ly_points(x=SalePrice, y = log(1 + xval_pred_svm$SalePrice), data=train, color="red", hover=Id) %>%
    ly_points(x=SalePrice, y = log(1 + xval_pred_rlm$SalePrice), data=train, color="green", hover=Id) %>%
    y_axis(label="SVM & RIDGE") %>%
    x_axis(label="IDEAL")

p2 <- figure(legend_location = "top_left", width=400) %>% 
  ly_points(x=log(1 + xval_pred_rlm$SalePrice), y = log(1 + SalePrice), data=xval_pred_svm, hover=Id) %>%
  ly_abline(a = 0, b = 1, width=2, type=2, color = "red", legend="Ideal") %>% x_axis(label="Ridge") %>% y_axis(label="SVM")

ptot <- grid_plot(list(p, p2), ncol = 2)
rbokeh2html(ptot,'./out.html')
IRdisplay::display_html(file='./out.html')

html file written to: ./out.html


<!DOCTYPE html>

We can see how **SVM** seems to be more **conservative**, in the sense that it predicts higher prices for the low price houses, and lower prices for the high price houses.

***

## <span style="color:#2E66A7"> Neural network ensemble</span>

The previous 2 model were pretty good in their own regards, but can we combine them to get something more powerful?  
And if so, how should we do it?

The approach presented here uses a **neural network** to combine the predictions of the previous 2 models, and spit out a final prediction that will be, hopefully, better than the single predictions.

We'll also see what happens if we simply take the **average** of the predictions, or the average of the **logarithms** of the predictions (which is less sensitive to outliers).

In [85]:
rm(list=ls())

library(readr)
library(purrr)
library(tidyr)
library(rbokeh)
library(plyr)
library(tidyr)
library(ggplot2)
library(moments)
library(sqldf)
library(xtable)
library(dplyr)
library(psych)
library(corrplot)
library(infotheo)
library(nnet)
library(Metrics)


#############
# READ DATA #
#############

# Import the data
train <- read_csv("../data/train_fin.csv")
test <- read_csv("../data/test_fin.csv")

train_svm <- read_csv("../data/predictions/svm_xval.csv")
train_rlm <- read_csv("../data/predictions/ridge_xval.csv")

# Create a training set by combining the predictions.
train_nn <- data.frame(Id=train$Id, svmSalePrice=train_svm$SalePrice, rlmSalePrice=train_rlm$SalePrice, SalePrice=train$SalePrice)

# Log-scale the prices.
train_nn$svmSalePrice <- log(1 + train_nn$svmSalePrice)
train_nn$rlmSalePrice <- log(1 + train_nn$rlmSalePrice)

all_data <- rbind(train[, -ncol(train)], test)

# Print header
print(head(train_nn))
# Count column types
table(sapply(train_nn, class))

Parsed with column specification:
cols(
  .default = col_double(),
  Id = col_integer()
)
See spec(...) for full column specifications.
Parsed with column specification:
cols(
  .default = col_double(),
  Id = col_integer()
)
See spec(...) for full column specifications.
Parsed with column specification:
cols(
  Id = col_integer(),
  SalePrice = col_double()
)
Parsed with column specification:
cols(
  Id = col_integer(),
  SalePrice = col_double()
)


  Id svmSalePrice rlmSalePrice SalePrice
1  1     12.25248     12.24289  12.24770
2  2     12.17304     12.27374  12.10902
3  3     12.30818     12.29455  12.31717
4  4     12.09515     12.03766  11.84940
5  5     12.59981     12.61487  12.42922
6  6     12.02276     12.03173  11.87061



integer numeric 
      1       3 

In [86]:
################
# PREPARE DATA #
################

# shuffle dataset
train_nn <- train_nn[sample(nrow(train_nn)),]

x_train = train_nn[, 2:ncol(train_nn)]
y_train = train_nn$SalePrice

num_folds = 10

# Print the initial values.
print(paste("SVM RMSE:", rmse(x_train$svmSalePrice, y_train)))
print(paste("RIDGE RMSE:", rmse(x_train$rlmSalePrice, y_train)))
# Just taking the mean seems better, but there is margin for improvement.
print(paste("AVERAGE RMSE:", rmse((x_train$rlmSalePrice + x_train$svmSalePrice)/2, y_train)))
print(paste("LOG-AVERAGE RMSE:", rmse(exp((log(x_train$rlmSalePrice) + log(x_train$svmSalePrice)) / 2) , y_train)))

# Save the predictions done by crossvalidation
xval_pred <- data.frame(Id=c(), SalePrice=c())

[1] "SVM RMSE: 0.11287785257848"
[1] "RIDGE RMSE: 0.112031988956003"
[1] "AVERAGE RMSE: 0.110301721294561"
[1] "LOG-AVERAGE RMSE: 0.110304763902576"


We can see that just by taking the **average** of the predictions we can sensibly reduce the **RMSE**.  

On the leaderboard, the **average** scores a value of $0.11991$, while the $log$**-average** gets $0.12004$.

Can a neural network do better than that simple formula?

**Note**: the following code takes a fair bit of time to run, do it at your own risk!

In [75]:
######
# NN #
######


index = sample(1:nrow(x_train))
fold_size <- floor(nrow(x_train) / num_folds)

params <- expand.grid(s = c(50, 100),
                      d = c(0.1, 0.2, 1))


for (i in 1:nrow(params))
{
  #############
  #############
  p <- params[i, ]
  
  cv_errors <- NULL
  for (i in 1:num_folds) {
    
    index_test <- (((i-1) * fold_size + 1):(i * fold_size))  
    if (i==num_folds)
    {
      index_test = c(index_test, ((i * fold_size)+1):nrow(x_train))
    }
    index_train <- setdiff(index, index_test)
    
    X_tr <- x_train[index_train, ]
    Y_tr <- y_train[index_train]  
    X_val <- x_train[index_test, ]  
    Y_val <- y_train[index_test] 
    
    # Scale x
    x_mean <- sapply(X_tr, mean)
    x_sd <- sapply(X_tr, sd)
    var_zero <- x_sd == 0
    X_tr <- data.frame(scale(X_tr, center = x_mean, scale = x_sd))
    X_tr[, var_zero] <- NULL
    X_val <- data.frame(scale(X_val, center = x_mean, scale = x_sd))
    X_val[, var_zero] <- NULL
    
    
    # Scale the output.
    Y_mean <- mean(Y_tr)
    Y_sd <- sd(Y_tr)
    Y_tr <- scale(Y_tr, center = Y_mean, scale = Y_sd)
    Y_val <- scale(Y_val, center = Y_mean, scale = Y_sd)
    
    
    model<- nnet(SalePrice ~ ., data = X_tr, size=p$s, linout=T, decay=p$d, maxit=3000, relrol=2^-12)
    
    pred <- predict(model, X_val)
    cv_errors <- c(cv_errors, rmse(pred * Y_sd + Y_mean, Y_val * Y_sd + Y_mean))
      
    # Scale back the prediction.
    pred <- pred * Y_sd + Y_mean
    pred <- exp(pred) - 1
    # Round to the closest 500
    pred <- round(pred / 500) * 500
    
    pred_data_temp <- data.frame(Id=train[index_test, ]$Id, SalePrice=pred)
    # Add the new predictions
    xval_pred <- rbind(xval_pred, pred_data_temp)
  }
  
  print(paste("SIZE=", p$s, "; DECAY=", p$d, "; Error=", round(mean(cv_errors), digits=4), " ; std dev=", round(sd(cv_errors), digits=4)))
  #############
  #############
}

xval_pred <- xval_pred %>% arrange(Id)
# write.csv(x=xval_pred, file="../data/predictions/nn_xval.csv", row.names=F)

# weights:  401
initial  value 20837.342738 
iter  10 value 103.479950
iter  20 value 99.002574
iter  30 value 97.741001
iter  40 value 97.276799
iter  50 value 96.967691
iter  60 value 96.688834
iter  70 value 96.491741
iter  80 value 96.376313
iter  90 value 96.288494
iter 100 value 96.238840
iter 110 value 96.210396
iter 120 value 96.191477
iter 130 value 96.183225
iter 140 value 96.177988
iter 150 value 96.173986
iter 160 value 96.171770
iter 170 value 96.170417
iter 180 value 96.169445
iter 190 value 96.168434
iter 200 value 96.167773
iter 210 value 96.167318
iter 220 value 96.166936
iter 230 value 96.166620
iter 240 value 96.166386
iter 250 value 96.166151
iter 260 value 96.165967
iter 270 value 96.165807
iter 280 value 96.165645
iter 290 value 96.165519
iter 300 value 96.165350
iter 310 value 96.165260
iter 320 value 96.165208
iter 330 value 96.165161
iter 340 value 96.165085
final  value 96.165009 
converged
# weights:  401
initial  value 6763.618893 
iter  10 value 103.394930


In [ ]:
###########
# PREDICT #
###########

test_svm <- read.csv("../data/predictions/svm_fin.csv")
test_rlm <- read.csv("../data/predictions/ridge_fin_01.csv")

# Write the log-mean to a file, it is also a good average.
log_res <-  exp((log(test_rlm$SalePrice) + log(test_svm$SalePrice)) / 2)
# Round to the closest 500
log_res <- round(log_res / 500) * 500
# write.csv(x=data.frame(Id=test$Id, SalePrice=log_res), file="../data/predictions/log_res.csv", row.names = F)

# Write the mean to a file, maybe it's better.
mean_res <- (test_rlm$SalePrice + test_svm$SalePrice) / 2
# Round to the closest 500
mean_res <- round(mean_res / 500) * 500
# write.csv(x=data.frame(Id=test$Id, SalePrice=mean_res), file="../data/predictions/mean_res.csv", row.names = F)


# Create a training set by combining the predictions.
x_test <- data.frame(svmSalePrice=test_svm$SalePrice, rlmSalePrice=test_rlm$SalePrice)

# Log-scale the prices.
x_test$svmSalePrice <- log(1 + x_test$svmSalePrice)
x_test$rlmSalePrice <- log(1 + x_test$rlmSalePrice)

# Scale x_train and x_test
Y_mean <- mean(x_train$SalePrice)
Y_sd <- sd(x_train$SalePrice)

x_mean <- sapply(x_train, mean)
x_sd <- sapply(x_train, sd)
var_zero <- x_sd == 0
x_train <- data.frame(scale(x_train, center = x_mean, scale = x_sd))
x_test <- data.frame(scale(x_test, center = x_mean[1:(ncol(x_train)-1)], scale = x_sd[1:(ncol(x_train)-1)]))
x_train[, var_zero] <- NULL
x_test[, var_zero] <- NULL

model<- nnet(SalePrice ~ ., data = x_train, size=100, linout=T, decay=0.1, maxit=3000, relrol=2^-14)

pred <- predict(model, x_test)


# Scale back the prediction.
pred <- pred * Y_sd + Y_mean
pred <- exp(pred) - 1
# Round to the closest 500
pred <- round(pred / 500) * 500

pred_data <- data.frame(Id=test$Id, SalePrice=pred)

# write.csv(x=pred_data, file="../data/predictions/nn_100.csv", row.names=F)

At last, we can compare the results we have found, and see if there is a statistically significant improvement over the previous results.

By the way, this model gives a score of $0.11979$, which is a bit better than the previous one, and enough to reach the top **20%**!

In [87]:
xval_pred_nn <- read.csv("../data/predictions/nn_xval.csv")
xval_pred_svm <- read.csv("../data/predictions/svm_xval.csv")
xval_pred_rlm <- read.csv("../data/predictions/ridge_xval.csv")

train_res = data.frame(RealPrice=train$SalePrice,
                       PriceSVM=log(1+xval_pred_svm$SalePrice),
                       PriceRidge=log(1+xval_pred_rlm$SalePrice),
                       PriceNN=log(1+xval_pred_nn$SalePrice))

train_res_2 <- train_res - train_res$RealPrice
train_res_2$RealPrice <- NULL
train_res_2 <- train_res_2^2

train_nn <- train_nn %>% arrange(Id)
p <- figure(legend_location = "top_left") %>% ly_points(x=train_nn$SalePrice, y = log(1 + xval_pred_nn$SalePrice), data=train_nn, hover=Id) %>%
  ly_abline(lm(log(1 + xval_pred_nn$SalePrice) ~ train_nn$SalePrice, data=train_nn), width=2, type=2, legend="Predicted") %>%
  ly_abline(a = 0, b = 1, width=2, type=2, color = "red", legend="Ideal")

rbokeh2html(p,'./out.html')
IRdisplay::display_html(file='./out.html')

html file written to: ./out.html


<!DOCTYPE html>

In [88]:
p <- figure() %>% ly_boxplot(X2, value, data=melt(train_res_2), outlier_glyph = NA)
rbokeh2html(p,'./out.html')
IRdisplay::display_html(file='./out.html')

html file written to: ./out.html


<!DOCTYPE html>

In [89]:
summary(lm(value~X2, data=melt(train_res_2)))


Call:
lm(formula = value ~ X2, data = melt(train_res_2))

Residuals:
     Min       1Q   Median       3Q      Max 
-0.01274 -0.01196 -0.00971 -0.00269  0.77489 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  0.0123917  0.0010748  11.529   <2e-16 ***
X2PriceRidge 0.0001595  0.0015200   0.105    0.916    
X2PriceSVM   0.0003498  0.0015200   0.230    0.818    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.04103 on 4368 degrees of freedom
Multiple R-squared:  1.215e-05,	Adjusted R-squared:  -0.0004457 
F-statistic: 0.02654 on 2 and 4368 DF,  p-value: 0.9738


On a side note, we cannot really say that the model are statistically different, as the differences of *square residuals* over the cross-validation predictions are all quite the same.